![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Taller: Construcción e implementación de modelos Bagging, Random Forest y XGBoost

En este taller podrán poner en práctica sus conocimientos sobre la construcción e implementación de modelos de Bagging, Random Forest y XGBoost. El taller está constituido por 8 puntos, en los cuales deberan seguir las intrucciones de cada numeral para su desarrollo.

## Datos predicción precio de automóviles

En este taller se usará el conjunto de datos de Car Listings de Kaggle donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Importación de librerías
%matplotlib inline
import pandas as pd
import numpy as np

# Lectura de la información de archivo .csv
data = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTrain_carListings.zip')

# Preprocesamiento de datos para el taller
data = data.loc[data['Model'].str.contains('Camry')].drop(['Make', 'State'], axis=1)
data = data.join(pd.get_dummies(data['Model'], prefix='M'))
data = data.drop(['Model'], axis=1)

# Visualización dataset
data.head()

,Price,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE
7,21995,2014,6480,False,False,False,True,False,False,False
11,13995,2014,39972,False,False,False,False,True,False,False
167,17941,2016,18989,False,False,False,False,False,True,False
225,12493,2014,51330,False,False,False,True,False,False,False
270,7994,2007,116065,False,True,False,False,False,False,False


In [7]:
# Separación de variables predictoras (X) y variable de interés (y)
y = data['Price']
X = data.drop(['Price'], axis=1)

In [42]:
print("Columnas en X:", X.columns)

Columnas en X: Index(['Year', 'Mileage', 'M_Camry', 'M_Camry4dr', 'M_CamryBase', 'M_CamryL',
       'M_CamryLE', 'M_CamrySE', 'M_CamryXLE'],
      dtype='object')


In [43]:
# Separación de datos en set de entrenamiento y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [44]:
print("Columnas en X_train:", X_train.columns)
print("Columnas en X_test:", X_test.columns)


Columnas en X_train: Index(['Year', 'Mileage', 'M_Camry', 'M_Camry4dr', 'M_CamryBase', 'M_CamryL',
       'M_CamryLE', 'M_CamrySE', 'M_CamryXLE'],
      dtype='object')
Columnas en X_test: Index(['Year', 'Mileage', 'M_Camry', 'M_Camry4dr', 'M_CamryBase', 'M_CamryL',
       'M_CamryLE', 'M_CamrySE', 'M_CamryXLE'],
      dtype='object')


### Punto 1 - Árbol de decisión manual

En la celda 1 creen un árbol de decisión **manualmente**  que considere los set de entrenamiento y test definidos anteriormente y presenten el RMSE y MAE del modelo en el set de test.

In [ ]:
# Celda 1


### Punto 2 - Bagging manual

En la celda 2 creen un modelo bagging **manualmente** con 10 árboles de regresión y comenten sobre el desempeño del modelo.

In [45]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor

# Celda 2
# Se crea un arreglo de 1 a 10
np.random.seed(1)

# Impresión de arreglo y muestreo aleatorio
nums = np.arange(1, 11)
print('Arreglo:', nums)
print('Muestreo aleatorio: ', np.random.choice(a=nums, size=10, replace=True))

Arreglo: [ 1  2  3  4  5  6  7  8  9 10]
Muestreo aleatorio:  [ 6  9 10  6  1  1  2  8  7 10]


In [46]:
# Creación de 10 muestras de bootstrap 
np.random.seed(123)

n_samples = X_train.shape[0]
n_B = 10

samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]
samples

[array([3582, 3454, 1346, ...,  826,  801, 5657]),
 array([6962, 3408, 2553, ..., 6611,  877, 6412]),
 array([1917, 3131,  384, ..., 2876, 6449, 6557]),
 array([3849, 4565, 6820, ..., 6835, 4643,  639]),
 array([2468, 3608, 1367, ..., 3108, 2961, 4357]),
 array([3142, 1537, 5966, ..., 3224, 6922, 3396]),
 array([6588, 3753, 1786, ..., 2131, 4627, 5672]),
 array([1289, 3776,  981, ..., 1891, 6034, 6165]),
 array([6079, 3819, 6976, ..., 6478, 2225, 5072]),
 array([3952, 3276,  896, ..., 4589, 1964,  893])]

In [52]:
# Visualización muestra boostrap #1 para entremiento
data.iloc[samples[0], :]

,Price,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE
137130,8995,2010,88909,False,False,False,False,True,False,False
132538,5998,2007,136022,False,False,False,False,True,False,False
51669,16500,2015,27217,False,False,False,False,False,True,False
154900,17508,2017,34279,False,False,False,False,False,True,False
199443,13695,2012,77117,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
10241,10790,2011,90375,False,False,False,False,True,False,False
4214,8995,2009,62991,False,True,False,False,False,False,False
32299,11988,2011,76862,False,False,False,False,True,False,False
31240,19248,2017,32465,False,False,False,False,False,True,False


In [48]:
X_test

,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE
257343,2012,62048,False,False,False,False,True,False,False
326011,2005,111565,False,True,False,False,False,False,False
242354,2014,24203,False,False,False,False,False,True,False
266376,2015,30475,False,False,False,False,False,False,True
396954,2014,30498,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...
144298,2016,39544,False,False,False,False,False,True,False
364521,2015,37109,False,False,False,False,False,False,True
120072,2017,27341,False,False,False,False,False,True,False
99878,2012,78959,False,True,False,False,False,False,False


In [55]:
# Construcción un árbol de decisión para cada muestra boostrap

from sklearn.tree import DecisionTreeRegressor

# Definición del modelo usando DecisionTreeRegressor de sklearn
treereg = DecisionTreeRegressor(max_depth=None, random_state=123)

# DataFrame para guardar las predicciones de cada árbol
y_pred = pd.DataFrame(index=X_test.index, columns=[list(range(n_B))])

# Entrenamiento de un árbol sobre cada muestra boostrap y predicción sobre los datos de test
for i, sample in enumerate(samples):
    X_train = data.iloc[sample, 1:]
    y_train = data.iloc[sample, 0]
    treereg.fit(X_train, y_train)
    y_pred.iloc[:,i] = treereg.predict(X_test)
    
y_pred

,0,1,2,3,4,5,6,7,8,9
257343,9900.0,9900.0,9900.0,9900.0,9900.0,11995.0,11788.0,9900.0,9900.0,9900.0
326011,5995.0,7495.0,5995.0,7495.0,5995.0,5995.0,6987.0,6700.0,7495.0,6492.0
242354,15814.0,15814.0,15814.0,15814.0,16491.0,15814.0,15814.0,15349.0,15814.0,15349.0
266376,17997.0,16691.0,15813.0,17997.0,17997.0,17997.0,17997.0,21990.0,23482.0,23482.0
396954,18995.0,18995.0,16951.0,16951.0,16951.0,16951.0,18995.0,15988.0,18995.0,18995.0
...,...,...,...,...,...,...,...,...,...,...
144298,15976.0,17988.0,15976.0,17988.0,17988.0,16977.0,15976.0,17988.0,15976.0,17988.0
364521,14699.0,14699.0,15700.0,16900.0,15700.0,15999.0,14995.0,14851.0,14851.0,14851.0
120072,18330.0,17803.0,18330.0,23533.0,18330.0,23533.0,18330.0,23533.0,18330.0,18330.0
99878,10495.0,10495.0,12989.0,12989.0,10495.0,10495.0,12989.0,10495.0,12995.0,10495.0


In [57]:
# Desempeño de cada árbol
from sklearn.metrics import mean_squared_error
for i in range(n_B):
    print('Árbol ', i, 'tiene un error: ', np.sqrt(mean_squared_error(y_pred.iloc[:,i], y_test)))

Árbol  0 tiene un error:  1660.1411726141453
Árbol  1 tiene un error:  1624.2014889152688
Árbol  2 tiene un error:  1619.3033008294965
Árbol  3 tiene un error:  1661.0335050691997
Árbol  4 tiene un error:  1642.5555994451854
Árbol  5 tiene un error:  1630.948425578028
Árbol  6 tiene un error:  1663.1370581249548
Árbol  7 tiene un error:  1651.6951167490276
Árbol  8 tiene un error:  1659.3293244606423
Árbol  9 tiene un error:  1635.898227033642


In [58]:
# Predicciones promedio para cada obserbación del set de test
y_pred.mean(axis=1)

257343    10298.3
326011     6664.4
242354    15788.7
266376    19144.3
396954    17876.7
           ...   
144298    17082.1
364521    15324.5
120072    19838.2
99878     11493.2
387162    12010.6
Length: 3464, dtype: object

In [59]:
# Error al promediar las predicciones de todos los árboles
np.sqrt(mean_squared_error(y_test, y_pred.mean(axis=1)))

1224.9754517796266

In [61]:
from sklearn.metrics import r2_score

# Suponiendo que y_test son los valores verdaderos y y_pred son las predicciones
r2 = r2_score(y_test, y_pred.mean(axis=1))

print("Coeficiente de determinación (R^2):", r2)

Coeficiente de determinación (R^2): 0.901899153010398


### Punto 3 - Bagging con librería

En la celda 3, con la librería sklearn, entrenen un modelo bagging con 10 árboles de regresión y el parámetro `max_features` igual a `log(n_features)` y comenten sobre el desempeño del modelo.

In [60]:
# Celda 3

import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor


max_features = int(np.log2(X.shape[1]))
base_estimator = DecisionTreeRegressor(max_features=max_features, random_state=42)
bagging_model = BaggingRegressor(base_estimator=base_estimator, n_estimators=10, random_state=42)
bagging_model.fit(X_train, y_train)
train_score = bagging_model.score(X_train, y_train)
test_score = bagging_model.score(X_test, y_test)
print("Desempeño del modelo de Bagging:")
print("Coeficiente de determinación (R^2) en el conjunto de entrenamiento:", train_score)
print("Coeficiente de determinación (R^2) en el conjunto de prueba:", test_score)

Desempeño del modelo de Bagging:
Coeficiente de determinación (R^2) en el conjunto de entrenamiento: 0.9830281756895175
Coeficiente de determinación (R^2) en el conjunto de prueba: 0.8500107406718443


### Punto 4 - Random forest con librería

En la celda 4, usando la librería sklearn entrenen un modelo de Randon Forest para regresión  y comenten sobre el desempeño del modelo.

In [ ]:
# Celda 4


### Punto 5 - Calibración de parámetros Random forest

En la celda 5, calibren los parámetros max_depth, max_features y n_estimators del modelo de Randon Forest para regresión, comenten sobre el desempeño del modelo y describan cómo cada parámetro afecta el desempeño del modelo.

In [ ]:
# Celda 5


### Punto 6 - XGBoost con librería

En la celda 6 implementen un modelo XGBoost de regresión con la librería sklearn y comenten sobre el desempeño del modelo.

In [ ]:
# Celda 6


### Punto 7 - Calibración de parámetros XGBoost

En la celda 7 calibren los parámetros learning rate, gamma y colsample_bytree del modelo XGBoost para regresión, comenten sobre el desempeño del modelo y describan cómo cada parámetro afecta el desempeño del modelo.

In [ ]:
# Celda 7


### Punto 8 - Comparación y análisis de resultados
En la celda 8 comparen los resultados obtenidos de los diferentes modelos (random forest y XGBoost) y comenten las ventajas del mejor modelo y las desventajas del modelo con el menor desempeño.

In [ ]:
# Celda 8
